# Global Coastal Transect System

Cross-shore coastal transects are essential to coastal monitoring, offering a consistent reference line to measure coastal change, while providing a robust foundation to map coastal characteristics and derive coastal statistics thereof. The Global Coastal Transect System consists of more than 11 million cross-shore coastal transects uniformly spaced at 100-m intervals alongshore, for all OpenStreetMap coastlines that are longer than 5 kilometers. The dataset is more extensively described Calkoen et al., 2024. "Enabling Coastal Analytics at Planetary Scale", that is currently in review. 

In [ ]:
import os

import dask

dask.config.set({"dataframe.query-planning": False})

import dask_geopandas
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

load_dotenv(override=True)

sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
account_name = "coclico"
storage_options = {"account_name": account_name, "credential": sas_token}

## Connect to the CoCliCo STAC 

In [ ]:
coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
gcts_collection = coclico_catalog.get_child("gcts")
gcts_collection

## The dataset is geospatially partitioned

In [ ]:
from coastpy.io.utils import read_items_extent

gcts_extents = read_items_extent(gcts_collection, columns=["geometry", "assets"])
gcts_extents[["geometry", "href"]].explore()

### Use a dynamic map to extract data by region of interest

The IPyleaflet map below can be used to find the bbox coordinates of a certain region.
Zoom to the area where you want to extract data and run the next cell. Please wait until the map is rendered; otherwise the coordinates cannot be extracted. 

In [ ]:
m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = 15.827, -95.96
m.zoom = 15
m.layout.height = "800px"
# m.layout.height = "725px"
# m.layout.width = "725px"
m



In [ ]:
west, south, east, north = m.west, m.south, m.east, m.north
# Note: small little hack to ensure the notebook also works when running all cells at once
if not west:
    west, south, east, north = (
        30.28415679931641,
        31.276790311057272,
        30.630912780761722,
        31.51123970051334,
    )
roi = gpd.GeoDataFrame(
    geometry=[shapely.geometry.box(west, south, east, north)], crs=4326
)

## Find the data partitions that span the region of interest

In [ ]:
hrefs = gpd.sjoin(gcts_extents, roi).href.to_list()

## Read the data from cloud storage

In [ ]:
transects = dask_geopandas.read_parquet(hrefs, storage_options=storage_options)
transects = (
    transects.sjoin(roi.to_crs(transects.crs)).drop(columns=["index_right"]).compute()
)

transects.head()

In [ ]:
import colorcet as cc

transects[["geometry", "bearing"]].hvplot(
    geo=True,
    tiles="ESRI",
    color="bearing",
    frame_width=650,
    frame_height=550,
    colorbar=True,
    cmap=cc.CET_C6,
    clim=(0,360),
    # title="Transect geometries with north bearing [deg]",
    clabel="North Bearing [deg]"
)